# Ödev Açıklaması:
- 5 Farklı Azerbaycanca dataseti Sentiment Analizi için temizleyeceğiz.

- Etiketler(labels) üç kutuplu ve sayısal olacak. => Negative=0.0, Neutral =0.5 Positive = 1.0

- Her dataset için 2 sütunlu Excel(xslx) üret: text,label featureları olucak

- Temizlediğimiz metinlerle Word2Vec ve FastText(gensim) modellerini eğit

- Pipeline hafif düzeyde alan(domain) farkındalığı içersin.(news,social,reviews,general)

- Kurulum ve tekrar çalıştırma kolay ve tekrarlanabilir olsun(seed,requirements,README)



# Teslim Edilmesi Gerekenler
1. Temizlenmiş data dosyaları:
    5 adet .xlsx(her biri 2 sütun feature olan text-label),UTF-8 uyumlu olmalı
2. Kodlar:
    - preprocess.py(temizleme + etiketleri sayıya çevirme + xlsx yazma)
    - train_embeddings.py(Word2Vec ve FastText eğitimi ve kaydetme)
3. Modeller:
    - models/word2vec_<domain>.model
    - models/fasttext_<domain>.model veya .bin
4. README.md:
    Adım adım çalıştırma,Python/kitaplık sürümleri,seed,ortalama eğitim süresi ve örnek komutlar
5. requirements.txt:
    - pandas, gensim, openpyxl, vb. sürümleri sabitle.



# Yapmamız gerekenler:
1. 5 veri kümesini temizle -> 5 excel dosyası üret
2. Bu Excel dosyalarındaki metinleri birleştir -> corpus_all.txt oluştur
3. Bu birleştirdiğimiz metinle (corpus_all.txt) ile iki modeli eğit: word2vec.model ve fasttext.model
4. Tümünü Githuba yükleyeceğiz:
    - data_clean/*.xlsx
    - corpus_all.txt
    - embeddings/word2vec.model
    - embeddings/fasttext.model
    - README.md (kısa rapor)

# Şimdi yapmaya başlayalım İlk yapacağımız iş Data Cleaning Verileri temizleyelim.
1. Excel labeled-sentiment.xlsx 
    Features: text,sentiment(3 tane sınıf var metinle girilmiş negative/neutral/positive) şekilde 
2. test__1_.xslx 
    Features text,label(label'lar genelde 0 ve 1 olarak verilmiş) 0=>negative 1 =>positive
3. train__3_xslx
    Features text,label(0 ve 1)
4. train-00000-of-00001.xlsx
    Features text,labels bu da metin olarak (positive/negative/neutral) olarak verilmiş
5. merged_dataset_CSV__1_xlsx
    Features text,labels(0,1)
    Bu datasette dikkat etmemiz gereken şey bazı excellerde otomatik eklenen Unnamed:* gibi bir index sütunu var Bunu almayacağız buna göre ele alacağız.

Label'lar için 
- negative/neg/-1/0 -> 0.0 yapacağız
- neutral/neu/0.5/ 1/2 / -> 0.5 yapacağız
- positive/pos/1/p -> 1.0 yapacağız
- Binary setlerde(neutral yok):Negatif örnekler 0.0 a pozitifler 1.0'a gider 


Excellerin herbirini ayrı ayrı şekilde temizleyeceğiz daha sonra da her birini data_clean/<dataadı>.xlsx şeklinde kaydedeceğiz.Excel sütunları cleaned_text ve sentiment_value şeklinde olacak.

Temiz Exceller hazır olunca bu 5 temiz Excel'i tek metin dosyasında toplayacağız: 
    Her satır: dom<domain> <cleaned sentence>
                Örnek:domnews sabah bakida yagis var 
    

# Hangi Teknolojileri kullanacağız ve Neden?
1. Python 3 -> Tüm scriptleri çalıştıracağımız ortam.
2. pandas -> Excel okuma/yazma, vektörize temizlik(hızlı str.replace,map vb.)
3. regex (re) -> URL,e-posta,@mention,#hashtag,sayılar gibi kalıpları yakalayıp temizlemek için 
4. unicodedata -> Azerbaycanca'ya duyarlı harf dönüşümü ve Unicode normalizasyonu(örn. farklı “İ/ı” varyantları, birleşik karakterler)
5. ftfy (opsiyonel) → Kırık/bozuk metin kodlamalarını düzeltmek için pratik (smart quotes, mojibake).
6. gensim → Word2Vec ve FastText eğitimleri.
7. openpyxl → pandas’ın .xlsx I/O motoru (Excel’le konuşan kütüphane).
8. scikit-learn (opsiyonel) → Sınıflandırma yapmayacağız ama istersen küçük sanity-check’ler, train/test ayırma, karıştırma vs. için işe yarar.


# Basit Temizlik Kuralları(Azericeye özel şeyler var burada)
1. Lowercase with Azerbaijani rules:
    replace İ->i , I->ı before lower() 
    Normal lower() fonksiyonu Türkçe ve Azerice de hatalı çalışır.Çünkü İ harfini i yerine ı yapar.
    Önce dönüşüm yapmamız gerek daha sonra lower yapmamız gerek.
    def az_lower(s):
    return s.replace("İ", "i").replace("I", "ı").lower()

2. REGEX'leri placeholderlar ile değiştir (URL's,emails,phone numbers, mentions):
    “URLs→URL, emails→EMAIL, phone numbers→PHONE, @mentions→USER”
    Yani metinde geçen özel yapıları yer tutucularla(placeholder) değiştir.
    Örnek: Bu gün www.news.az saytında gördüm → Bu gün URL saytında gördüm  
           Əlaqə:  test@mail.az → Əlaqə: EMAIL  
           Zəng et: 050-123-45-67 → Zəng et: PHONE  
           @aygun yazdı → USER yazdı

3. Strip HTML tags/entities;drop but keep hastag text:
     "#" işaretini kaldır ama hastag içeriğini koru, camelCase hastagleri böl (örn:#BakuCityLife -> baku city life)

4. Collapse ≥3 repeated letters to 2
    Üç veya daha fazla aynı harf arka arkaya gelirse ikiye indir.
    Böylece abartılı yazımın etkisi korunur ama gürültü azalır.
    coooool -> coool 

5. Replace digits with <NUM>
    Sayıları "<NUM>" etiketi ile değiştir.

6. Remove standalone punctuation and extra spaces
    Yalnız noktalama işaretleriyle oluşan tokenleri kaldır.
    Fazla boşlukları teke indir.
    Şu harfleri korumamız gerek: ə, ğ, ı, ö, ü, ç, ş (ve x, q)

7. Remove single-letter tokens except o, e
    Yani “a”, “u”, “t” gibi tek harfli kelimeleri kaldır ama “o” ve “e” kalsın.
    o mənim → o mənim  
    a çox yaxşı → çox yaxşı

8. Drop exact duplicates and empty rows
    Aynı metin satırı birden fazla varsa tekini tut,boş satır varsa kaldır.
    

# Mini Challenges Kısmı
1. Hastag Split(örnek:#QarabagIsBack → qarabag is back)
    Amaç: camelCase/TitleCase hashtag’i kelimelere bölmek, # işaretini atmak.
    Bu kısmı clean_text yapmadan önce lowercase yapmandan önce çağırmak iyi olur; sonra az-lover ile küçültme yaparız

2. Emoji mapping (tokenization’dan önce küçük eşleme)
    Amaç: Emojileri EMO_POS /EMO_NEG gibi sabitlere çevirmek 
    EMOJI_MAP = {
    "😀":"EMO_POS","😃":"EMO_POS","😄":"EMO_POS","😁":"EMO_POS","😊":"EMO_POS","🙂":"EMO_POS","😍":"EMO_POS","👍":"EMO_POS","❤️":"EMO_POS","🥳":"EMO_POS",
    "😞":"EMO_NEG","😠":"EMO_NEG","😡":"EMO_NEG","😢":"EMO_NEG","😭":"EMO_NEG","👎":"EMO_NEG","☹️":"EMO_NEG","🤬":"EMO_NEG"
    }
    def map_emojis(s: str) -> str:
        return ''.join(EMOJI_MAP.get(ch, ch) for ch in s)

3. Stopword Araştırması(Azeri+başka bir dil)
    İstek: Azerice stopword’leri başka bir dille (TR/EN/RU) karşılaştır; 10 aday öner (negasyonları atma!).

        Atılmaması gereken negasyonlar: yox, deyil, heç, qətiyyən, yoxdur (model için önemli sinyal).

        Önerilen 10 aday (domaine göre güncellenebilir):

            1. və (ve), 2. amma (ama), 3. ki, 4. bu, 5. həm,

            2.hər, 7. bütün, 8. ilə (ile), 9. kimi, 10. belə

        Not: “da/de” (Azeri’de da, də) da güçlü aday; ayrıca mı/mi karşılığı parçacıklar (mı/mi yok ama mı, mi benzeri sorma ekleri Azerice’de farklı işler). Küçük bir pilot EDA ile en sık 50 kelimeden domain-spesifik 10 seçebilirsin.

4. Negation scope (toggle): negator’dan sonra 3 token işaretle 
    İstek: Negasyon kelimesi görülürse sonraki 3 token’a _NEG eki ekle.
    
    Örnek:“yox xoşuma gəldi” → “yox xoşuma_NEG gəldi_NEG …”

    Değerlendirme fikri: Bu işaretleme açık/kapalıyken Word2Vec’te “xoşuma_NEG” gibi tokenlerin en yakın komşuları nasıl değişiyor? Küçük örnek tabloyla nitel kıyas yap.

5. Simple deasciify (ör.: cox→çox, yaxsi→yaxşı)
    Amaç:ASCII yazılmış Azeri karakterlerini basit haritayla düzeltmek.
    “Raporda kaç token değişti?” sorusu için changed sayısını yaz.
